<a href="https://colab.research.google.com/github/CodeCraftIA/telethon_scrape/blob/main/%CE%91%CE%BD%CF%84%CE%AF%CE%B3%CF%81%CE%B1%CF%86%CE%BF_%CF%84%CE%BF%CF%85_fiverr_telegram_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Install the library



In [ ]:
!pip install telethon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.8/599.8 kB 7.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for telethon: filename=Telethon-1.35.0-py3-none-any.whl size=675107 sha256=f855eb15def8a1e7ab55de1d5f44316e8ae716104c1c3e2bc6e87aacfcbf01b1
  Stored in directory: /root/.cache/pip/wheels/7b/18/81/4a5a3adfd4af392db743a5a5942157c220bbedda57a17d6bbf
  Created wheel for pyaes: filename=pyaes-1.6.1-py3-none-any.whl size=26347 sha256=d10c47dcaac67cea2fba732f8d50d625b50b49771e7090310211d9e0207f8de8
  Stored in directory: /root/.cache/pip/wheels/d6/84/5f/ea6aef85a93c7e1922486369874f4740a5642d261e09c59140
Successfully built telethon pyaes


CHANGE THE DATA ON THE NEXT CELL TO YOUR DATA, API_ID - API_HASH and the target CHANNEL-GROUP

In [ ]:
# Initialize your client here
api_id = 'xxx'
api_hash = 'xxxxx'

You need to create a txt file with all the group-channels handles you want the script to check, save them in channels.txt

Scrape participants of a public group

In [ ]:
from telethon.sync import TelegramClient
from datetime import datetime, timedelta
import asyncio
import pandas as pd

async def scrape_participants(channels):
    try:
        async with TelegramClient('anon', api_id, api_hash) as client:
            for channel in channels:
                try:
                  data = []
                  async for participant in client.iter_participants(channel):
                      if participant.first_name:
                          participant_name = f"{participant.first_name}"
                      else:
                          participant_name = ""
                      data.append({
                          'ID': participant.id,
                          'Username': participant.username,
                          'Name': participant_name,
                          'Link': f'https://web.telegram.org/a/#{participant.id}'
                      })
                except Exception as e:
                    print(f"Oops, something went wrong while scraping participants from '{channel}'")
                    continue  # Continue to the next channel even if an exception occurs

            # Convert data to DataFrame
            df = pd.DataFrame(data)

            # Define the file name
            file_name = 'participants.xlsx'

            # Export DataFrame to Excel file
            df.to_excel(file_name, index=False)
            print(f"Participants scraped successfully from '{channel}' and saved in '{file_name}'.")
    except Exception as e:
        print("Oops, something went wrong while scraping participants.")

async def scrape_messages(channels):
    try:
        async with TelegramClient('anon', api_id, api_hash) as client:
            for channel in channels:
                try:
                  data_messages = []
                  async for message in client.iter_messages(channel, limit=100):
                      # Convert datetime to timezone-unaware datetime
                      date_unaware = message.date.replace(tzinfo=None)
                      # Get media if message has any
                      media_url = None
                      if message.media:
                          if hasattr(message.media, 'photo'):
                              media_url = message.media.photo

                          elif hasattr(message.media, 'document'):
                              media_url = message.media.document
                      data_messages.append({
                          'ID': message.id,
                          'Date': date_unaware,
                          'Text': message.text,
                          'Media URL': media_url,
                      })
                except Exception as e:
                    print(f"Oops, something went wrong while scraping participants from '{channel}'")
                    continue  # Continue to the next channel even if an exception occurs

            # Convert data to DataFrame
            df = pd.DataFrame(data_messages)

            # Define the file name
            file_name = 'messages.xlsx'

            # Export DataFrame to Excel file
            df.to_excel(file_name, index=False)
            print(f"Messages scraped successfully from '{channel}' and saved in '{file_name}'.")
    except Exception as e:
        print("Oops, something went wrong while scraping messages.")

# Read channels from the file
with open('channels.txt', 'r') as file:
    channels = file.read().splitlines()

# Use await directly in the notebook to run the async functions
await scrape_participants(channels)
await scrape_messages(channels)

Participants scraped successfully from 'benofmpublicgroup' and saved in 'benofmpublicgroup_participants.xlsx'.
Oops, something went wrong while scraping messages.
